# Python script for automating the procedure of Service Provider validation in Varient URLs

Developed by [Ali Fazeli](ali.fazeli@huawei.com) 

#### for your information

What this module does is it selects host addresses from the 'test' excel file, filters it and it will pass it into **'who_is_this()'** module which validates SP (service provider) of requested host by achive and processing Whois request from [IANA](www.iana.org). 

In the rest of this notebook, when we mention 'query' we mean 'Whois query'

#### script dependencies

- **_pythonwhois_** module
- **openpyxl** module

## First Stage : making a whois request with python and retrieve a customized respond in order to achieving Service Provider information from owner of DNS

### Printing out the whois information of an IP

In [ ]:
# importing stuff
import pythonwhois
import socket
import re

In [2]:
some_web_site = pythonwhois.get_whois('zoomit.ir')

In [3]:
# here we can see what's the query information look like
# it will return the whole information
some_web_site

{'expiration_date': [datetime.datetime(2022, 8, 25, 0, 0)],
 'updated_date': [datetime.datetime(2017, 6, 26, 0, 0)],
 'nameservers': ['marek.ns.cloudflare.com', 'val.ns.cloudflare.com'],
 'emails': ['info@samansystems.com'],
 'contacts': {'registrant': {'handle': 'na1407-irnic',
   'name': 'Najme Ataeinejad',
   'email': 'hitking@gmail.com',
   'city': 'Tehran',
   'state': 'Tehran',
   'country': 'IR',
   'phone': '02122122588',
   'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek-e-Gharb'},
  'tech': {'handle': 'na1407-irnic',
   'name': 'Najme Ataeinejad',
   'email': 'hitking@gmail.com',
   'city': 'Tehran',
   'state': 'Tehran',
   'country': 'IR',
   'phone': '02122122588',
   'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek-e-Gharb'},
  'admin': {'handle': 'na1407-irnic',
   'name': 'Najme Ataeinejad',
   'email': 'hitking@gmail.com',
   'city': 'Tehran',
   'state': 'Tehran',
   'country': 'IR',
   'phone': '02122122588',
   'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek

In [4]:
# invoking name of owner, in a most tidy format
print(some_web_site['contacts']['registrant'])

{'handle': 'na1407-irnic', 'name': 'Najme Ataeinejad', 'email': 'hitking@gmail.com', 'city': 'Tehran', 'state': 'Tehran', 'country': 'IR', 'phone': '02122122588', 'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek-e-Gharb'}


#### accessing specific values of whois information 

In [5]:
# printing out what sub-methods "some_web_site" have
# some_web_site.__dir__()

In [6]:
# what data we have in our query
some_web_site.keys()

dict_keys(['expiration_date', 'updated_date', 'nameservers', 'emails', 'contacts', 'raw'])

#### expiration date of Domain

In [7]:
temp_data = some_web_site.get('expiration_date')

In [8]:
temp_data[0].strftime('Domain expiration date is: %d, %b %Y')

'Domain expiration date is: 25, Aug 2022'

#### getting contents of our query 

In [9]:
some_website_info_contents = some_web_site.get('contacts')


In [10]:
# registrant part of contacts
some_website_info_contents['registrant']

{'handle': 'na1407-irnic',
 'name': 'Najme Ataeinejad',
 'email': 'hitking@gmail.com',
 'city': 'Tehran',
 'state': 'Tehran',
 'country': 'IR',
 'phone': '02122122588',
 'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek-e-Gharb'}

---

to 'registrar' or 'registrant', that is the question...

In [11]:
str(some_web_site.get('registrar'))

'None'

In [12]:
str(some_web_site.get('registrant'))

'None'

** Most important point should be mention here is that strucure of queries differs a lot.**

submitted websites (DNSs) have different kind of data stuctures for presenting information about owner of the website. so best decision is searching for important information in the whole query using specific keyword and filtering it in order to achieve relevant information about owner of website (which we gonna pass as Service Provider of processed website).   

___

In [13]:
# tech part of contacts
some_website_info_contents['tech']

{'handle': 'na1407-irnic',
 'name': 'Najme Ataeinejad',
 'email': 'hitking@gmail.com',
 'city': 'Tehran',
 'state': 'Tehran',
 'country': 'IR',
 'phone': '02122122588',
 'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek-e-Gharb'}

In [14]:
# admin part of contacts
some_website_info_contents['admin']

{'handle': 'na1407-irnic',
 'name': 'Najme Ataeinejad',
 'email': 'hitking@gmail.com',
 'city': 'Tehran',
 'state': 'Tehran',
 'country': 'IR',
 'phone': '02122122588',
 'street': 'No 14\nSepehr St\nFarahzadi Av\nShahrek-e-Gharb'}

## Creating a function to do search about Owner or Acount Responsible of requested Website in Whois Query

### ** who_is_this function ** , main core of script


In [23]:
__author__ = "Ali Fazeli"
__email__ = "ali.fazeli@huawei.com"
__status__ = "developing"


def pass_domain_as_sp(orphan_url):
    if re.search(string=orphan_url, pattern=r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$"):
        # this is a ip address and cannot be truncated
        pass
    else:
        company = orphan_url.split(".")[-2]
        return company


def who_is_this(domain):
    
    findings = []
   
    try:
        # fire it up! getting a whois query from invoked domain
        whois_things = pythonwhois.get_whois(domain)
        
        # trying to find out the owner of website  
        
        try:                    
            if whois_things['registrar'] != None:
                print(whois_things['registrar'])      
                findings.append(whois_things['registrar'])
        except (KeyError,TypeError) as e:
            #print ('{} not found.'.format(str(e)))          
            pass
        
        try:                    
            if whois_things['registrant'] != None:
                print(whois_things['registrant'])      
                findings.append(whois_things['registrant'])
        except (KeyError,TypeError) as e:
            #print ('{} not found.'.format(str(e)))
            pass
        
        
        try:
            if whois_things['contacts']['registrar']['name'] != None:
                print(whois_things['contacts']['registrant']['name']) 
                findings.append(whois_things['contacts']['registrant']['name'])
        except (KeyError,TypeError) as e:
            #print ('{} not found.'.format(str(e)))
            pass
        
        
        try:
            if whois_things['contacts']['registrant']['name'] != None:
                print(whois_things['contacts']['registrant']['name'])
                findings.append(whois_things['contacts']['registrant']['name'])
        except Exception as e:
            #print ('{} not found.'.format(str(e)))
            pass
        
        
        try:
            if whois_things['contacts']['tech']['organization'] != None :
                print(whois_things['contacts']['tech']['organization'])
                findings.append(whois_things['contacts']['tech']['organization'])
        except (KeyError,TypeError) as e:
            #print ('{} not found.'.format(str(e)))
            pass
        
        
        try:
            if whois_things['contacts']['organization'] != None :
                print(whois_things['contacts']['organization'])
                findings.append(whois_things['contacts']['organization'])
        except (KeyError,TypeError) as e:
            #print ('{} not found.'.format(str(e)))
            pass
        
        # bye-bye
        except Exception:
            print ('\nThere seems to be no registrant for this domain.')
            #company = domain
            pass
        
        
        # spliting the input and lowering letters
        splitup = domain.lower().split()
        # making a pattern from input
        patern = re.compile('|'.join(splitup))
        
        while True:
            # base on domain feature of whois query...
            # if we have domain information about requested query...
            if patern.search(domain):
                #print('Whois Results Are Good ' + company)
                print ('\nThe Whois Results Look Promising: {} '.format(domain))
                
                # making sure if it is okay...
                # TO-DO : change this with like method in order to automate the interactive asking process 
                accept = input('\nIs the  Search Term sufficient?: '.lower())
                if accept in ('y', 'yes'): 
                    pass_domain_as_sp(domain)
                    break
                # just in case if there is no appropriate company name, we can overwrite the currect value, or not xD
                elif accept in ('n', 'no'):
                    #
                    # in case of fiiling company name...
                    print('User Supplied Company ' + company)
                    company = temp_company
                    break
                else:
                    print ('\nThe Options Are yes|no Or y|no Not {}'.format(accept))
 
            else:
                # if we don't have domain information about requested query...
                #print('Whois Results Not Good ' + company)
                print ("\n\tThe Whois Results Don't Look Very Promissing: '{}'".format(domain))
                temp_company = input('\nRegistered Company Name: ')
                if temp_company == '':
                    print('User Supplied Blank Company')
                    # in the case of leaving "temp_company" blank domain value will be assigned to company value
                    company = domain
                    break
                else:
                    # in case of fiiling company name...
                    print('User Supplied Company ' + company)
                    company = temp_company
                    break
 
    # if we cannot perform "pythonwhois.get_whois(domain)" thing...
    except pythonwhois.shared.WhoisException:
        pass
    # this exception is raised for socket-related errors
    except socket.error:
        pass
    # if the key we are requesting is not in the query information we've retrieving
    except KeyError:
        pass
    # if we have network problem with proxies and stuff like that...
    except pythonwhois.net.socket.errno.ETIMEDOUT:
        print ('\nWhoisError: You may be behind a proxy or firewall preventing whois lookups. Please supply the registered company name, if left blank the domain name ' + '"' + domain + '"' +' will be used for the Linkedin search. The results may not be as accurate.')
        temp_company = input('\nRegistered Company Name: ')
        if temp_company == '':
            company = domain
        else:
            company = temp_company
    except Exception:      
        print("An Unhandled Exception Has Occured, sry:((")
        
    # if we don't have any "company" info in our query...
    # "locals()" method looks up in the query dictionary
    if 'company' not in locals():
        print ('There is no Whois data for this domain.\n\nPlease supply a company name.')
        pass
    
        """
        while True:
            temp_company = input('\nRegistered Company Name: ')
            if temp_company == '':
                # in the case of leaving "temp_company" blank domain value will be assigned to company value
                print('User Supplied Blank Company')
                company = domain
                break
            else:
                # in case of fiiling company name...
                company = temp_company
                print('User Supplied Company ' + company)
                break
         """
    print(company)
    return company

In [24]:
# providing some input
# the example deliberately has to many dots for testing the truncating of URL
domain = 'webex.com'

In [ ]:
# who_is_this() module testing 
who_is_this(domain)

## Second Stage : Reading data from an Ecxel file and passing it into Pandas Data Frames

In [4]:
from openpyxl import load_workbook
import pandas as pd
import re

In [5]:
# reading excel file using panda's read_excel module
excel_file = pd.read_csv("test.csv")

In [ ]:
# showing up first three records of data
excel_file.loc[:5]

In [ ]:
# selecting host and sp feature 
df = pd.DataFrame(excel_file, columns = ['host', 'sp'])

In [ ]:
type(df)

In [ ]:
df.head()

### Filtering the 'host' feature and passing it to who_is_this() module

in this section, we filter unappropriate host data in order of standardization

- deleting the sub-domains of host addresses which have sub-domains and keeping the main domain of website
- ignoring IPv6 hosts (which logically they are wrong records, just because they haven't any DNS set on)
- ignoring any host addresses which have space (Unicode Character 'SPACE' (U+0020)) (we have significant count of records in this category)
- removing **https//** from beginning of address 
- removing **/** (slash) from the end of file

In [ ]:
# defining a list for temporarily store new SP suggestions, in order to append it to the Data-Frame at the end of who_is_this() function

new_SP_values = []

for domain in df['host']:
    #use domain for completing new sp (service provider) column
    # domains with redundent sub-domains
    if domain[0:2]=="*.":
        domain = domain[2:]
        new_SP_values.append(who_is_this(domain))
    # we're gonna ignore ipv6
    elif re.search(string=domain, pattern=r"[A-F0-9]{1,4}\:[A-F0-9]{0,4}\:[A-F0-9]{0,4}\:[A-F0-9]{0,4}\:[A-F0-9]{0,4}\:[A-F0-9]{0,4}\:[A-F0-9]{0,4}\:[A-F0-9]{0,4}"):
        new_SP_values.append("")
        continue
    # hosts which have space, we're gonna ignore them
    elif re.search(string=domain, pattern=r"\s"):  
        new_SP_values.append("")
        continue
    # deleting slash at the end 
    elif re.search(string=domain, pattern=r"\/$"):
        domain = domain[:-1]
    # getting rid of http and http of beginning
    elif re.search(string=domain, pattern=r"^http:\/\/"):
        domain = domain[7:]
    elif re.search(string=domain, pattern=r"^https:\/\/"):
        domain = domain[8:]
    else:
        # very standard hosts will go to the heaven
        # calling the who_is_this() module
        # who_is_this(domain)
        
        # append the processed query result to our temporarily list 
        new_SP_values.append(who_is_this(domain))
        
# save temporarily list which contains query results in the main Pandas Data-Frame
df['new_SP'] = new_SP_values

In [76]:
df.head()

,host,sp,new_SP
0,2A03:2880:F21C:81C4:FACE:B00C:0000:43FE,instagram,
1,dc2aio.iranlms.ir,rubika,iranlms
2,*.fevn1-1.fna.fbcdn.net,instagram,fbcdn
3,uupload.ir,NaN,uupload
4,dl1.sarzamindownload.com,NaN,sarzamindownload


### Advantages of script above

we can filter a lot of records which are un-standard Host Addresses. for instance **IPv6** addresses with no DNS recording to that and **un-supported Host addresses which have space in their address** and other static patterns which could apply to filter records (and subsequently report the miss-indexed records by A**).

** as you can see, the hosts which doesn't look like a standard Host are filtered and obviously didn't pass to who_is_this function.**

## Writing the data into a new column in Pandas Data-Frame

### saving data in SP column in pre-loaded Excel file 

**TO-DO** : overwriting the data into preloaded Excel file

In [77]:
writer = pd.ExcelWriter('output.xlsx')

In [78]:
df.to_excel(writer,'Sheet1',)
# df2.to_excel(writer,'Sheet2')

In [79]:
writer.save()